In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrap_imdb_highest_grossing(num_pages=1):
    """
    Scrap de las primeras `num_pages` páginas de la lista de IMDb 'Top 1000 Highest-Grossing Movies'.
    Devuelve un DataFrame con la información de cada película.
    """
    base_url = "https://www.imdb.com/list/ls098063263/"
    all_movies = []

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/112.0.0.0 Safari/537.36"
        )
    }

    for page in range(1, num_pages + 1):
        # Construimos la URL con la paginación
        url = f"{base_url}?page={page}"
        print(f"Scrapeando página {page}: {url}")

        # Hacemos la petición
        response = requests.get(url, headers=headers)
        # Parseamos el HTML
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Seleccionamos el contenedor principal de la lista de películas
        movie_list = soup.select_one("#__next > main > div > section > div > section > div > div.sc-3e58c83b-1.cixVoM.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div:nth-child(2) > ul")
        
        # Cada ítem de película está en un <li> con clase "ipc-metadata-list-summary-item"
        movie_items = movie_list.find_all("li", class_="ipc-metadata-list-summary-item") if movie_list else []
        
        for item in movie_items:
            # Título
            title_tag = item.find("h3", class_="ipc-title__text")
            title = title_tag.get_text(strip=True) if title_tag else None
            
            # Año, duración, clasificación, etc. (a menudo aparecen en un contenedor con clase "dli-title-metadata")
            metadata_spans = item.find_all("span", class_="sc-d5ea4b9d-7")
            # Dependiendo de cómo estén ordenados esos <span>, a veces: [Año, Duración, Clasificación]
            year = metadata_spans[0].get_text(strip=True) if len(metadata_spans) > 0 else None
            duration = metadata_spans[1].get_text(strip=True) if len(metadata_spans) > 1 else None
            cert = metadata_spans[2].get_text(strip=True) if len(metadata_spans) > 2 else None
            
            # Rating de IMDb (número de estrellas)
            rating_span = item.find("span", class_="ipc-rating-star--rating")
            rating = rating_span.get_text(strip=True) if rating_span else None
            
            # Número de votos (a veces está en un span con clase "ipc-rating-star--voteCount")
            votes_span = item.find("span", class_="ipc-rating-star--voteCount")
            votes = votes_span.get_text(strip=True) if votes_span else None

            # Recaudación (la muestra un <span> o <div> con texto "Worldwide Lifetime Gross: ...")
            gross_div = item.find("div", class_="ipc-html-content-inner-div")
            # A veces te tocará hacer un .find(...) más específico o buscar con regex "Worldwide Lifetime Gross"
            # Para este ejemplo, asumimos que contiene un texto con "Worldwide Lifetime Gross: $xxx"
            gross_value = None
            if gross_div and "Worldwide Lifetime Gross:" in gross_div.get_text():
                text = gross_div.get_text()
                # Ejemplo de extraer la cifra tras "Worldwide Lifetime Gross:"
                # Podríamos usar una pequeña lógica o regex para quedarnos con la parte del $
                try:
                    gross_value = text.split("Worldwide Lifetime Gross:")[1].strip()
                except IndexError:
                    gross_value = None
            
            # Enlace a la página de la película
            link_tag = item.find("a", class_="ipc-lockup-overlay")
            partial_link = link_tag['href'] if link_tag else None
            # Añadimos la parte base de IMDb si es un enlace relativo
            movie_url = f"https://www.imdb.com{partial_link}" if partial_link else None
            
            # Guardamos en una lista
            all_movies.append({
                "title": title,
                "year": year,
                "duration": duration,
                "certification": cert,
                "rating": rating,
                "votes": votes,
                "gross": gross_value,
                "movie_url": movie_url
            })
        
        # Ponemos un pequeño delay para no saturar el servidor
        time.sleep(2)
    
    # Convertimos todo a un DataFrame para mayor comodidad
    df_movies = pd.DataFrame(all_movies)
    return df_movies

# Ejecución de la función
if __name__ == "__main__":
    df = scrap_imdb_highest_grossing(num_pages=4)  # Por ejemplo, scrapea las primeras 2 páginas
    print(df.head(10))
    # Podrías exportar a CSV
    df.to_csv("../data/imdb_top1000_highest_grossing.csv", index=False)

Scrapeando página 1: https://www.imdb.com/list/ls098063263/?page=1
Scrapeando página 2: https://www.imdb.com/list/ls098063263/?page=2
Scrapeando página 3: https://www.imdb.com/list/ls098063263/?page=3
Scrapeando página 4: https://www.imdb.com/list/ls098063263/?page=4
Scrapeando página 5: https://www.imdb.com/list/ls098063263/?page=5
Scrapeando página 6: https://www.imdb.com/list/ls098063263/?page=6
Scrapeando página 7: https://www.imdb.com/list/ls098063263/?page=7
Scrapeando página 8: https://www.imdb.com/list/ls098063263/?page=8
                                     title  year duration certification  \
0                                1. Avatar  2009   2h 42m             7   
1                   2. Vengadores: Endgame  2019    3h 1m             7   
2           3. Avatar: El sentido del agua  2022   3h 12m            12   
3                               4. Titanic  1997   3h 14m            13   
4  5. Star Wars: El despertar de la Fuerza  2015   2h 18m             7   
5             